<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Разбиение-данных-на-выборки" data-toc-modified-id="Разбиение-данных-на-выборки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Разбиение данных на выборки</a></span></li><li><span><a href="#Исследование-моделей" data-toc-modified-id="Исследование-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование моделей</a></span><ul class="toc-item"><li><span><a href="#Исследование-модели-решающего-дерева" data-toc-modified-id="Исследование-модели-решающего-дерева-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Исследование модели решающего дерева</a></span></li><li><span><a href="#Исследование-модели-случайного-леса" data-toc-modified-id="Исследование-модели-случайного-леса-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Исследование модели случайного леса</a></span></li><li><span><a href="#Исследование-модели-логистической-регрессии" data-toc-modified-id="Исследование-модели-логистической-регрессии-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Исследование модели логистической регрессии</a></span></li></ul></li><li><span><a href="#Проверка-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-лучшей-модели-на-тестовой-выборке-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка лучшей модели на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Рекомендация тарифов

**Цель проекта:**  
построить для оператора мобильной связи «Мегалайн» систему, способную проанализировать поведение клиентов и предложить пользователям, пользующимся архивными тарифами, на замену новый тариф - «Смарт» или «Ультра».
- В проекте используются данные о поведении клиентов, которые уже перешли на эти тарифы. 
- В процессе проекта будет построена модель для задачи классификации, которая выберет подходящий тариф.
- При построении модели будет необходимо достичь максимально большого значения *accuracy*, не менее 0.75.

# Импорт библиотек и данных

Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

Импорт и обзор данных

In [2]:
# Импорт данных
try:
    df = pd.read_csv('users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')
print(df.shape)
print(df.info())
display(df.head()) 

(3214, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбиение данных на выборки

In [3]:
# разделение обучающей и испытательной выборки в пропорции 3:2
df_train, df_trial = train_test_split(df, test_size=0.4, random_state=12345)
# разделение испытательной выборки на проверочную и тестовую в пропорции 1:1
df_valid, df_test = train_test_split(df_trial, test_size=0.5, random_state=12345) 
# создание переменных для признаков и целевого признака
features_train = df_train.drop('is_ultra', axis=1) #обучающая выборка
target_train = df_train['is_ultra']
features_valid = df_valid.drop('is_ultra', axis=1) #проверочная выборка
target_valid = df_valid['is_ultra'] 
features_test = df_test.drop('is_ultra', axis=1) #тестовая выборка
target_test = df_test['is_ultra'] 
# вывод размеров выборок для проверки
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


**Вывод:**
проведен импорт и обзор данных, а также их разбиение на обучающую, валидационную и тестовую выборки в пропорции 3:1:1

## Исследование моделей

### Исследование модели решающего дерева

In [4]:
best_model = None
best_result = 0

# проверка эффективности модели в зависимости от гиперпараметра max_depth в промежутке от 1 до 10

for depth in range(1, 10): # цикл с перебором различной глубины дерева
    
    #обучение модели с заданной глубиной дерева
    
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) # обучение модели на тренировочной выборке
    predictions = model.predict(features_valid) # получение предсказания модели
    result = accuracy_score(target_valid, predictions) # определение качества модели на валидационной выборке
    if result > best_result: # выбор наилучшего результата и соответствующих ему гиперпараметров модели
        best_model = model # сохранение наилучшей модели
        best_result = result #  сохранение наилучшего значения метрики accuracy на валидационных данных
        best_depth = depth # сохранение наилучшей глубины дерева
        
print("Accuracy наилучшей модели:", best_result)
print('Лучшая глубина дерева',best_depth)

Accuracy наилучшей модели: 0.7853810264385692
Лучшая глубина дерева 3


### Исследование модели случайного леса

In [5]:
best_model = None
best_result = 0

# проверка эффективности модели в зависимости от гиперпараметра n_estimators в промежутке от 1 до 16

for est in range(1, 16): # цикл с перебором различного количества деревьев
    
    #обучение модели с заданным количеством деревьев
    
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) # обучение модели на тренировочной выборке
    predictions = model.predict(features_valid) # получение предсказания модели
    result = accuracy_score(target_valid,predictions) # определение качества модели на валидационной выборке
    if result > best_result: # выбор наилучшего результата и соответствующих ему гиперпараметров модели
        best_model = model# сохранение наилучшей модели
        best_result = result #  сохранение наилучшего значения метрики accuracy на валидационных данных
        best_est = est # сохранение наилучшего количества деревьев

print("Accuracy наилучшей модели:", best_result)
print('Лучшее количество деревьев',best_depth)

Accuracy наилучшей модели: 0.7869362363919129
Лучшее количество деревьев 3


### Исследование модели логистической регрессии

In [6]:
best_model = None
best_result = 0

#проверка эффективности модели в зависимости от гиперпараметра max_iter в промежутке от 1 до 16

for iter in range(100, 500, 10): # цикл с перебором различного количества итераций обучения
    
    #обучение модели с заданным количеством итераций
    
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iter) 
    model.fit(features_train,target_train) # обучение модели на тренировочной выборке
    predictions = model.predict(features_valid) # получение предсказания модели
    result = accuracy_score(target_valid,predictions) # определение качества модели на валидационной выборке
    if result > best_result: # выбор наилучшего результата и соответствующих ему гиперпараметров модели
        best_model = model # сохранение наилучшей модели
        best_result = result #  сохранение наилучшего значения метрики accuracy на валидационных данных
        best_max_iter = iter # сохранение наилучшего количества итераций обучения

print("Accuracy наилучшей модели:", best_result)
print('Лучшее количество итераций обучения',best_max_iter)

Accuracy наилучшей модели: 0.7107309486780715
Лучшее количество итераций обучения 100


**Вывод:**  
Было проведено исследование для разных гиперпараметров на обучающей и валидационной выборке трёх моделей классификации:  
- модель решающего дерева
- модель случайного леса
- модель логистической регрессии 

Наилучший результат (аccuracy) =  0.7869 показала модель случайного леса с количеством деревьев 3.  
Именно она и признана лучшей моделью в данном проекте.

## Проверка лучшей модели на тестовой выборке

In [7]:
model = RandomForestClassifier(random_state=12345, n_estimators=3)#обучение модели с количеством деревьев 3
model.fit(features_train,target_train) # обучение модели на тренировочной выборке
predictions_test = model.predict(features_test)# получение предсказания модели на тестовой выборке
accuracy_test = accuracy_score(target_test, predictions_test) # определение качество модели на валидационной выборке
print("Accuracy")
print("Лучшая модель, тестовая выборка:", accuracy_test)

Accuracy
Лучшая модель, тестовая выборка: 0.7573872472783826


**Вывод:**  
Лучшая модель показала на тестовой выборке результат (accuracy) 0.7574, что лишь незначительно уступает результату 0.7869 этой же модели на валидационной выборке.  
Этот результат выше целевого показателя 0.75, следовательно, данная модель успешно прошла проверку на тестовой выборке. 

## Проверка модели на адекватность

Для проверки на адекватность использовалась модель DummyClassifier с гиперпараметром strategy="most_frequent", когда модель всегда возвращает наиболее часто используемую метку класса в целевом признаке, переданном для обучения. Если результат (accuracy) лучшей модели будет больше, чем у DummyClassifier, то лучшая модель будет признана адекватной. 

In [8]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train,target_train)
#DummyClassifier(strategy='most_frequent')
predictions_dummy = dummy_clf.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_dummy) # определение качество модели на валидационной выборке
print("Accuracy")
print("Модель Dummy, тестовая выборка:", accuracy_test)

Accuracy
Модель Dummy, тестовая выборка: 0.6842923794712286


**Вывод:**  
Лучшая модель случайного леса с количеством деревьев 3 показала результат (accuracy) 0.7574, что превышает результат модели DummyClassifier 0.6843, следовательно лучшая модель прошла проверку на адекватность.

## Общий вывод

**По проекту было выполнено следующее:**
- импорт и обзор данных
- разбиение данных на обучающую, валидационную и тестовую выборки в пропорции 3:1:1
- исследование на обучающей и валидационной выборке трёх моделей:
    - модель решающего дерева
    - модель случайного леса
    - модель логистической регрессии
- выбор лучшей модели и её проверка на тестовой выборке и на адекватность.

**Выводы:**
- лучшей моделью по результатам исследования признана модель случайного леса с количеством деревьев 3
- результат данной модели по точности предсказания (accuracy_score) на тестовой выборке составил 0.7574 что превышает целевой показатель 0.75
- данная модель успешно прошла проверку на адекватность в с равнениии с моделью DummyClassifier.

**Рекомендации сотрудникам компании оператора мобильной связи «Мегалайн»:** 
- построенную в рамках данного проекта систему анализа поведения клиентов можно применять к абонентам, пользующимся архивными тарифами, и она будет с точностью не менее 0.75 предлагать им на замену новый тариф - «Смарт» или «Ультра».